## Reference
   - https://huggingface.co/airesearch/wangchanberta-base-att-spm-uncased
   - https://huggingface.co/docs/transformers/index
   - https://simpletransformers.ai/docs/ner-specifics/

In [1]:
import dill
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Prepair Data

In [2]:
path_name = "../../dataset/data/"

with open(path_name + 'biased-pos.data', 'rb') as file:
    datatofile = dill.load(file)

tagged_sents = []
for data in datatofile:
    text_inside = []
    for word, pos, label in data:
        if word.strip() == '':
            text_inside.append(('_', label))
        else:
            text_inside.append((word, label))
    tagged_sents.append(text_inside)
# print(tagged_sents[0])
train_sents, test_sents = train_test_split(tagged_sents, test_size=0.2, random_state=42)
print(len(train_sents))
print(len(test_sents))
print(train_sents[1])

400
100
[('ศาสตราจารย์', 'O'), ('ไมเคิล', 'O'), ('_', 'O'), ('เจ', 'O'), ('.', 'O'), ('_', 'O'), ('ไท', 'O'), ('เลอ', 'O'), ('ร์', 'O'), ('_', 'O'), ('(', 'O'), ('เกิด', 'O'), ('_', 'O'), ('พ.ศ.', 'O'), ('_', 'O'), ('2480', 'O'), (')', 'O'), ('_', 'O'), ('หรือ', 'O'), ('ที่', 'O'), ('รู้จัก', 'O'), ('กัน', 'O'), ('ทั่วไป', 'O'), ('ใน', 'O'), ('ชื่อ', 'O'), ('_', 'O'), ('""', 'O'), ('ไมค์', 'O'), ('_', 'O'), ('ไท', 'O'), ('เลอ', 'O'), ('ร์""', 'O'), ('_', 'O'), ('หรือ', 'O'), ('ที่', 'O'), ('เรียกกันว่า', 'O'), ('_', 'O'), ('""', 'O'), ('ชาย', 'O'), ('กบ', 'O'), ('""', 'O'), ('_', 'O'), ('เป็น', 'O'), ('นักวิชาการ', 'O'), ('ที่', 'B-d'), ('มีชื่อเสียง', 'I-d'), ('ใน', 'O'), ('ออสเตรเลีย', 'O'), ('ใต้', 'O'), ('_', 'O'), ('ตั้ง', 'O'), ('ข้อสังเกต', 'O'), ('ใน', 'O'), ('งานวิจัย', 'O'), ('ของ', 'O'), ('เขา', 'O'), ('เกี่ยวกับ', 'O'), ('กบ', 'O'), ('และ', 'O'), ('คางคก', 'O'), ('_', 'O'), ('โดยส่วนใหญ่', 'O'), ('อยู่', 'O'), ('ที่', 'O'), ('มหาวิทยาลัย', 'O'), ('แอ', 'O'), ('ดิ', 'O'), ('

In [3]:
def convert_to_simple_transformer_format(sentences):
    sentence_id = []
    words = []
    labels = []

    for idx, sents in enumerate(sentences):
        for word, label in sents:
            label = label.upper().replace("-", "_")
            sentence_id.append(idx)
            words.append(word)
            labels.append(label)
    return pd.DataFrame(
        {"sentence_id": sentence_id, "words": words, "labels": labels}
    )    
            


In [4]:
train_data = convert_to_simple_transformer_format(train_sents)
test_data = convert_to_simple_transformer_format(test_sents)
train_data

,sentence_id,words,labels
0,0,kevin,O
1,0,_,O
2,0,mitnick,O
3,0,_,O
4,0,เป็น,O
...,...,...,...
11054,399,และ,O
11055,399,ครู,O
11056,399,สอน,O
11057,399,ทฤษฎี,O


# Train

In [5]:
import torch
from simpletransformers.ner import NERModel, NERArgs

_NER_TAGS = ["O", "B_D", "I_D", "B_E", "I_E"]

# Configure the model
ner_args = NERArgs()
ner_args.evaluate_during_training = True
ner_args.overwrite_output_dir = True
ner_args.train_batch_size = 16
ner_args.max_seq_length = 128
ner_args.num_train_epochs = 100 #10
ner_args.save_model_every_epoch = False

model = NERModel(
    "bert", "monsoon-nlp/bert-base-thai", args=ner_args, use_cuda=torch.cuda.is_available(), labels=_NER_TAGS
)

# Train the modelk
model.train_model(train_data,eval_data=test_data)

Some weights of the model checkpoint at monsoon-nlp/bert-base-thai were not used when initializing BertForTokenClassification: ['cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized

  0%|          | 0/1 [00:00<?, ?it/s]

C:\Users\kitti\.conda\envs\bias_py37\lib\site-packages\transformers\optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/100 [00:00<?, ?it/s]

Running Epoch 0 of 100:   0%|          | 0/25 [00:00<?, ?it/s]

...

Running Evaluation:   0%|          | 0/13 [00:00<?, ?it/s]

Running Epoch 99 of 100:   0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/13 [00:00<?, ?it/s]

(2500,
 defaultdict(list,
             {'global_step': [25,
               50,
               75,
               100,
               125,
               150,
               175,
               200,
               225,
               250,
               275,
               300,
               325,
               350,
               375,
               400,
               425,
               450,
               475,
               500,
               525,
               550,
               575,
               600,
               625,
               650,
               675,
               700,
               725,
               750,
               775,
               800,
               825,
               850,
               875,
               900,
               925,
               950,
               975,
               1000,
               1025,
               1050,
               1075,
               1100,
               1125,
               1150,
               1175,
              

# Train Continue

In [ ]:
import torch
from simpletransformers.ner import NERModel, NERArgs

_NER_TAGS = ["O", "B_D", "I_D", "B_E", "I_E"]

ner_args = NERArgs()
ner_args.train_batch_size = 32
ner_args.evaluate_during_training = True
ner_args.overwrite_output_dir = True
ner_args.max_seq_length = 256
ner_args.num_train_epochs = 100 #10

model_continue = NERModel("camembert", 'outputs/checkpoint-1820-epoch-91', args=ner_args, use_cuda=torch.cuda.is_available(), labels=_NER_TAGS)
model_continue.train_model(train_data,eval_data=test_data)